In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
import time
import mysql.connector

data_list = [] 
# 웹 드라이버 설정 (예: Chrome)
driver = webdriver.Chrome()

# 1. 페이지 로드
url = "https://www.kebhana.com/cms/rate/index.do?contentUrl=/cms/rate/wpfxd651_07i.do#//HanaBank"
driver.get(url)

# 2. 페이지 로드 대기
wait = WebDriverWait(driver, 10)

state = "EUR"
# 3. 로딩 레이어가 사라질 때까지 대기
try:
    loading_layer = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#OPB_loadingLayerID_generatedByJSOPB_modalMaskID_generatedByJS")))
    wait.until(EC.invisibility_of_element(loading_layer))
except TimeoutException:
    print("Loading layer did not disappear, continuing...")

# 4. "기간환율변동" 선택 클릭
try:
    period_exchange_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "label[for='inqType_p'].radioForm")))
    period_exchange_button.click()
except (TimeoutException, ElementClickInterceptedException):
    driver.execute_script("document.querySelector(\"label[for='inqType_p']\").click();")

# 5. 날짜 수정 (조회 기간 시작일)
date_input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#tmpInqStrDt_p")))
date_input.clear()
date_input.send_keys("2023-08-29")

# 6. 날짜 수정 (조회 기간 종료일)
date_input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#tmpInqEndDt_p")))
date_input.clear()
date_input.send_keys("2024-09-13")

# 7. 통화 선택 (예: USD)
currency_select = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#curCd")))
select = Select(currency_select)
select.select_by_value(state)

# 8. "최종" 라디오 버튼 선택
try:
    final_radio_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "label[for='tmpPbldDvCd_2']")))
    final_radio_button.click()
except (TimeoutException, ElementClickInterceptedException):
    driver.execute_script("document.querySelector(\"label[for='tmpPbldDvCd_2']\").click();")

# 9. "조회" 버튼 클릭
query_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#HANA_CONTENTS_DIV > div.btnBoxCenter > a")))
query_button.click()

# 10. 조회 후 5초 대기
time.sleep(5)

# 11. 테이블 데이터 크롤링
rows = driver.find_elements(By.CSS_SELECTOR, "#searchContentDiv > div.printdiv > table > tbody > tr")
for row in rows:
    columns = row.find_elements(By.TAG_NAME, "td")
    data = [column.text for column in columns]
    
    # 첫 번째 값 제외, 나머지를 float로 변환
    for i in range(1, len(data)):
        try:
            data[i] = float(data[i].replace(',', ''))  # 쉼표 제거 후 float 변환
        except ValueError:
            print(f"변환 실패: {data[i]}")
    data_list.append(data)
    print("Row:", data)

# 웹 드라이버 종료
driver.quit()

# MySQL 연결 정보 설정
config = {
    'user': 'yangjinan',
    'password': 'ALsk85200!!',
    'host': 'hanaex.mysql.database.azure.com',
    'database': 'my_database',
    'port': 3306
}

# MySQL 연결
conn = mysql.connector.connect(**config)
cursor = conn.cursor()

# 데이터 삽입을 위한 SQL 쿼리
insert_query = """
INSERT INTO ex_table (
    state, date, cash_buy, cash_sell, remit_send, remit_receive, 
    foreign_currency_check_sell, exchange_rate, compared_to_previous, 
    exchange_fee_rate, usd_conversion_rate
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# data_list의 각 데이터를 데이터베이스에 삽입
for data in data_list:
    try:
        # 데이터 매핑
        date = data[0]  # 날짜
        cash_buy = data[1]  # 현찰 사실 때
        cash_sell = data[2]  # 현찰 파실 때
        remit_send = data[3]  # 송금 보내실 때
        remit_receive = data[4]  # 송금 받으실 때
        foreign_currency_check_sell = data[5]  # 여행자수표 파실 때
        exchange_rate = data[6]  # 매매 기준율
        compared_to_previous = data[7]  # 대비
        usd_conversion_rate = data[8]  # US 달러 재정환율
        exchange_fee_rate = data[9]  # 환가료율

        # SQL 쿼리 실행
        cursor.execute(insert_query, (
            state,
            date,
            cash_buy,
            cash_sell,
            remit_send,
            remit_receive,
            foreign_currency_check_sell,
            exchange_rate,
            compared_to_previous,
            exchange_fee_rate,
            usd_conversion_rate
        ))
        # 데이터 입력
    except Exception as e:
        print(f"Error inserting data {data}: {e}")

# 변경사항 커밋
conn.commit()

# 커서와 연결 종료
cursor.close()
conn.close()

Row: ['2024-09-13', 1508.72, 1449.86, 1494.08, 1464.5, 1462.28, 1479.29, 3.59, 5.419, 1.1122]
Row: ['2024-09-12', 1512.38, 1453.38, 1497.7, 1468.06, 1465.82, 1482.88, 6.39, 5.464, 1.1075]
Row: ['2024-09-11', 1505.87, 1447.11, 1491.25, 1461.73, 1459.49, 1476.49, 5.62, 5.485, 1.1014]
Row: ['2024-09-10', 1511.6, 1452.62, 1496.93, 1467.29, 1465.03, 1482.11, 0.74, 5.496, 1.1024]
Row: ['2024-09-09', 1510.84, 1451.9, 1496.18, 1466.56, 1464.3, 1481.37, 3.8, 5.517, 1.1039]
Row: ['2024-09-06', 1514.72, 1455.62, 1500.02, 1470.32, 1468.05, 1485.17, 1.78, 5.511, 1.1087]
Row: ['2024-09-05', 1512.9, 1453.88, 1498.22, 1468.56, 1466.28, 1483.39, 2.08, 5.536, 1.1112]
Row: ['2024-09-04', 1510.78, 1451.84, 1496.12, 1466.5, 1464.22, 1481.31, 0.11, 5.563, 1.1084]
Row: ['2024-09-03', 1510.67, 1451.73, 1496.01, 1466.39, 1464.11, 1481.2, 1.41, 5.564, 1.1045]
Row: ['2024-09-02', 1512.11, 1453.11, 1497.43, 1467.79, 1465.5, 1482.61, 3.08, 5.577, 1.1073]
Row: ['2024-08-30', 1508.97, 1450.09, 1494.32, 1464.74, 1462

Row: ['2024-05-08', 1496.97, 1438.57, 1482.44, 1453.1, 1450.73, 1467.77, 4.89, 5.827, 1.0745]
Row: ['2024-05-07', 1491.99, 1433.77, 1477.5, 1448.26, 1445.89, 1462.88, 0.56, 5.84, 1.0756]
Row: ['2024-05-03', 1492.56, 1434.32, 1478.07, 1448.81, 1446.44, 1463.44, 7.3, 5.848, 1.0769]
Row: ['2024-05-02', 1500.0, 1441.48, 1485.44, 1456.04, 1453.66, 1470.74, 13.56, 5.833, 1.0727]
Row: ['2024-04-30', 1513.83, 1454.77, 1499.14, 1469.46, 1467.06, 1484.3, 8.82, 5.823, 1.0717]
Row: ['2024-04-29', 1504.84, 1446.12, 1490.23, 1460.73, 1458.34, 1475.48, 0.92, 5.837, 1.0723]
Row: ['2024-04-26', 1503.9, 1445.22, 1489.3, 1459.82, 1457.44, 1474.56, 0.75, 5.827, 1.0693]
Row: ['2024-04-25', 1504.66, 1445.96, 1490.06, 1460.56, 1458.19, 1475.31, 1.13, 5.79, 1.073]
Row: ['2024-04-24', 1503.51, 1444.85, 1488.92, 1459.44, 1457.07, 1474.18, 1.72, 5.794, 1.0698]
Row: ['2024-04-23', 1501.76, 1443.16, 1487.18, 1457.74, 1455.37, 1472.46, 2.48, 5.812, 1.0701]
Row: ['2024-04-22', 1499.23, 1440.73, 1484.67, 1455.29, 145

Row: ['2023-12-27', 1466.71, 1409.49, 1452.48, 1423.72, 1421.39, 1438.1, 5.95, 5.851, 1.1105]
Row: ['2023-12-26', 1460.64, 1403.66, 1446.47, 1417.83, 1415.51, 1432.15, 2.71, 5.84, 1.1042]
Row: ['2023-12-22', 1463.41, 1406.31, 1449.2, 1420.52, 1418.2, 1434.86, 5.64, 5.834, 1.1012]
Row: ['2023-12-21', 1457.66, 1400.78, 1443.51, 1414.93, 1412.62, 1429.22, 1.99, 5.828, 1.0994]
Row: ['2023-12-20', 1455.63, 1398.83, 1441.5, 1412.96, 1410.65, 1427.23, 3.91, 5.834, 1.0945]
Row: ['2023-12-19', 1459.61, 1402.67, 1445.45, 1416.83, 1414.52, 1431.14, 7.02, 5.828, 1.0975]
Row: ['2023-12-18', 1452.45, 1395.79, 1438.36, 1409.88, 1407.58, 1424.12, 0.02, 5.819, 1.0917]
Row: ['2023-12-15', 1452.43, 1395.77, 1438.34, 1409.86, 1407.57, 1424.1, 2.71, 5.808, 1.0921]
Row: ['2023-12-14', 1449.67, 1393.11, 1435.6, 1407.18, 1404.88, 1421.39, 1.94, 5.837, 1.0993]
Row: ['2023-12-13', 1447.69, 1391.21, 1433.64, 1405.26, 1402.97, 1419.45, 0.36, 5.833, 1.0877]
Row: ['2023-12-12', 1448.06, 1391.56, 1434.0, 1405.62, 14